In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
path = "/content/drive/MyDrive/fNIRS_Deeplearning_Test"
os.chdir(path)
os.listdir(path)
# path_dataset='./dataset/bandpass_30subjects/'
path_dataset='./dataset/30subjects/'

In [10]:
import sys

path_to_this_work = '/content/drive/MyDrive/fNIRS_Deeplearning_Test'
path_to_src = path_to_this_work + '/src'

sys.path.insert(0, path_to_src)
os.environ['PYTHONPATH'] += (":"+path_to_src)
from BCI_model_class import SCCNet, SCCNet_25s, ShallowConvNet, ShallowConvNet2, cus_EEGNet
import Dataset_training_schema as dts
import train_model as tm

In [11]:
!pip install einops

In [12]:
from other_model import NIRS_CNN,NIRS_ANN, NIRS_LSTM, LSTM_getDataLoader
from transformer import Residual, PreNorm, FeedForward, Attention, Transformer, PreBlock, fNIRS_T, LabelSmoothing, train_transformer

In [13]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import scipy
from scipy import io
import numpy as np
from torch.utils.data import Dataset, DataLoader
from os import listdir
from os.path import isfile, isdir, join
import re
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import KFold, StratifiedKFold

In [14]:
from torchsummary import summary
import torch
import time

In [ ]:
# !pip install wandb -qqq
# import wandb
# wandb.login(key="550fa288f18a4938b6519ecd67d11309cfd9d51e")

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [16]:
def dataloader_model_train(test, train, config, model=0, rep_ID=""):
    channel_num = train.HbO.shape[1]
    n_Hb=1

    if config.Hb=='HbO':
        print("Hb type : HbO")
        train_loader = dts.getDataLoader(train.HbO, train.labels, batch_size=config.batch_size, onehot_encoding=False)
        test_loader = dts.getDataLoader(test.HbO, test.labels, batch_size=config.batch_size, onehot_encoding=False)
    elif config.Hb=='HbR':
        print("Hb type : HbR")
        train_loader = dts.getDataLoader(train.HbR, train.labels, batch_size=config.batch_size, onehot_encoding=False)
        test_loader = dts.getDataLoader(test.HbR, test.labels, batch_size=config.batch_size, onehot_encoding=False)
    elif config.Hb=='HbT':
        print("Hb type : HbT")
        train_loader = dts.getDataLoader(train.HbT, train.labels, batch_size=config.batch_size, onehot_encoding=False)
        test_loader = dts.getDataLoader(test.HbT, test.labels, batch_size=config.batch_size, onehot_encoding=False)
    elif config.Hb=='HbO+HbR_40channel':
        print("Hb type : HbO+HbR_40channel")
        Hb_test = np.concatenate((np.expand_dims(test.HbO,1), np.expand_dims(test.HbR,1)), axis=2)
        Hb_train = np.concatenate((np.expand_dims(train.HbO,1), np.expand_dims(train.HbR,1)), axis=2)
        train_loader = dts.getDataLoader(Hb_train, train.labels, batch_size=32, onehot_encoding=False)
        test_loader = dts.getDataLoader(Hb_test, test.labels, batch_size=32, onehot_encoding=False)
        channel_num = channel_num*2
    elif config.Hb=='HbO+HbR_2layer':
        print("Hb type : HbO+HbR_2layer")
        Hb_test = np.concatenate((np.expand_dims(test.HbO,1), np.expand_dims(test.HbR,1)), axis=1)
        Hb_train = np.concatenate((np.expand_dims(train.HbO,1), np.expand_dims(train.HbR,1)), axis=1)
        print(Hb_train.shape)
        train_loader = dts.getDataLoader(Hb_train, train.labels, batch_size=32, onehot_encoding=False)
        test_loader = dts.getDataLoader(Hb_test, test.labels, batch_size=32, onehot_encoding=False)
        n_Hb=2

    my_lr = config.learning_rate if model==0 else config.learning_rate/10
    my_epoch = config.epochs if model==0 else math.ceil(config.epochs/10)
    if model==0:
      # model = cus_EEGNet(config.kernel_size)
      if config.architecture=="EEGNet":
          # model = cus_EEGNet(conv1_size=config.kernel_size, n_Hb=1, time_point=train_HbO.shape[2], channel_num =20,  acti_fun='elu')
          model = cus_EEGNet(conv1_size=config.kernel_size, sep_conv_size=config.sep_conv_size, n_Hb=n_Hb, time_point=train.HbO.shape[2], channel_num=channel_num, acti_fun=config.acti_fun,  pooling_type = config.pooling_type, class_num=test.labels.shape[1], con1_type=config.con1_type)
          model=model.to(device)
          summary(model, (n_Hb, channel_num, train.HbO.shape[2]))
      elif config.architecture=="NIRS_CNN":
          model = NIRS_CNN(Hb_num=n_Hb, ch=train.HbO.shape[1], time_point=train.HbO.shape[2], class_num=train.labels.shape[1])
          model=model.to(device)
          print("channel_num: ",channel_num,"-------------------------------------------------------------------------------------------------------------------------")
          print("train_HbO.shape[1]: ",train.HbO.shape[1],"-------------------------------------------------------------------------------------------------------------------------")
          print("train_HbO.shape[2]: ",train.HbO.shape[2],"-------------------------------------------------------------------------------------------------------------------------")
          summary(model, (n_Hb, channel_num, train.HbO.shape[2]))
      elif config.architecture=="NIRS_transformer":
          model = fNIRS_T(n_class=train.labels.shape[1], sampling_point= train.HbO.shape[2], dim=128, depth=6, heads=8, mlp_dim=64).to(device)
      elif config.architecture=="NIRS_ANN": #*****************************************************************************************************************************
          # train_HbO = np.squeeze(train_HbO)
          # test_HbO = np.squeeze(test_HbO)
          # train_loader = dts.getDataLoader(train_HbO, train_labels, batch_size=16, onehot_encoding=False)
          # test_loader = dts.getDataLoader(test_HbO, test_labels, batch_size=16, onehot_encoding=False)
          model = NIRS_ANN(Hb_num=n_Hb, ch=train.HbO.shape[1], time_point=train.HbO.shape[2], class_num=train.labels.shape[1])
          model=model.to(device)
          summary(model, (n_Hb, train.HbO.shape[1], train.HbO.shape[2]))
          # summary(model, (1, train_HbO.shape[1]))
      elif config.architecture=="NIRS_LSTM":
          # model = FNIRSLSTM(input_size, hidden_size, num_classes)
          model = NIRS_LSTM()
          train_HbO_lstm = train.HbO.transpose(0, 2, 1)
          test_HbO_lstm = test.HbO.transpose(0, 2, 1)
          train_loader = LSTM_getDataLoader(train_HbO_lstm, train.labels, batch_size=8, onehot_encoding=False)
          test_loader = LSTM_getDataLoader(test_HbO_lstm, test.labels, batch_size=8, onehot_encoding=False)
      else:
          if config.duration=='10s':
              if config.architecture=="SCCNet":
                  model = SCCNet(config.kernel_size)
              elif config.architecture=="ShallowConvNet":
                  model = ShallowConvNet()
              model=model.to(device)
              summary(model, (1, 20, 134))
          elif config.duration=='25s':
              if config.architecture=="SCCNet":
                  model = SCCNet_25s(config.kernel_size)
              elif config.architecture=="ShallowConvNet":
                  model = ShallowConvNet()
              model=model.to(device)
              summary(model, (1, 20, 334))



    if config.architecture=="NIRS_transformer":
        return train_transformer(model,train_loader,test_loader,sub=config.sub,epoch=my_epoch,rep_ID = rep_ID)
    else:
        return tm.test_EEG_kernel(train_loader,test_loader,
                model=model,
                optimizer=config.optimizer,
                # kernel_size=config.kernel_size,
                epoch=my_epoch,
                learning_rate=my_lr,
                weight_decay=config.weight_decay,
                wandb_import=False,
                rep_ID = rep_ID)

In [17]:
class config:
    def __init__(self,Hb,acti_fun,architecture,batch_size,con1_type,dataset,duration,epochs,kernel_size,learning_rate,optimizer,pooling_type,scheme,sep_conv_size,sub,weight_decay):
        self.Hb = Hb
        self.acti_fun = acti_fun
        self.architecture = architecture
        self.batch_size = batch_size
        self.con1_type = con1_type
        self.dataset = dataset
        self.duration = duration
        self.epochs = epochs
        self.kernel_size = kernel_size
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.pooling_type = pooling_type
        self.scheme = scheme
        self.sep_conv_size = sep_conv_size
        self.sub = sub
        self.weight_decay = weight_decay

In [31]:
import math
def train(config):
  path_dataset='./dataset/'+ config.dataset + '_' + config.duration + '/'
  s1=path_dataset+'S'
  s2='.mat'



  if config.scheme == 'SI' or config.scheme == 'SIFT': #------------------modify
      if (config.dataset=='MA') | (config.dataset=='MI') | (config.dataset=='MA_raw') | (config.dataset=='MI_raw'):
        # test_HbO, test_HbR, test_HbT, test_labels, train_HbO, train_HbR, train_HbT, train_labels = dts.leave_subject_out(s1,s2, data_range=range(1,30), test_id=int(config.sub))
        test, train = dts.leave_subject_out(s1,s2, data_range=range(1,30), test_id=int(config.sub))
      else:
        test, train = dts.leave_subject_out(s1,s2, data_range=range(1,31), test_id=int(config.sub))
  elif config.scheme == 'SD':
      if (config.dataset=='MA') | (config.dataset=='MI') | (config.dataset=='MA_raw') | (config.dataset=='MI_raw'):
        range1 = []
        range2 = list(range(0,60,1))
      else:
        range1 = []
        range2 = list(range(0,75,1))

      test, train = dts.leave_trial_out(s1, s2, int(config.sub), range1, range2)
  print(train.HbO.shape)

  if config.scheme == 'SI' : #------------------modify
      model, criterion, optimizer = dataloader_model_train(test, train, config)
  elif config.scheme == 'SD':
      skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=None)
      skf.get_n_splits()
      print(skf) # StratifiedKFold(n_splits=2, random_state=None, shuffle=False)
      # get train set and test set index
      for rep_ID, (train_index, test_index) in enumerate(skf.split(train.HbO, np.argmax(train.labels, axis=1))):
          #print("train_index: ",train_index)
          print(rep_ID)
          print("test_index: ",test_index)
          sd_test = dts.fnirs_data(HbO = train.HbO[test_index], HbR = train.HbR[test_index], HbT = train.HbT[test_index], labels = train.labels[test_index])
          sd_train = dts.fnirs_data(HbO = train.HbO[train_index], HbR = train.HbR[train_index], HbT = train.HbT[train_index], labels = train.labels[train_index])
          dataloader_model_train(sd_test, sd_train, config, rep_ID=str(rep_ID))
  elif config.scheme == 'SIFT':
      # model, criterion, optimizer = dataloader_model_train(test_HbO, test_HbR, test_HbT, test_labels, train_HbO, train_HbR, train_HbT, train_labels, config)
      model, criterion, optimizer = dataloader_model_train(test, train, config)

      skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=None)
      skf.get_n_splits()
      print(skf) # StratifiedKFold(n_splits=2, random_state=None, shuffle=False)
      # get train set and test set index
      for rep_ID, (val_index, trainFT_index) in enumerate(skf.split(test.HbO, np.argmax(test.labels, axis=1))):
          print("trainFT_index: ",trainFT_index)
          FT_trainFT = dts.fnirs_data(HbO = train.HbO[trainFT_index], HbR = train.HbR[trainFT_index], HbT = train.HbT[trainFT_index], labels = train.labels[trainFT_index])
          FT_val = dts.fnirs_data(HbO = train.HbO[val_index], HbR = train.HbR[val_index], HbT = train.HbT[val_index], labels = train.labels[val_index])
          dataloader_model_train(FT_val, FT_trainFT, config, model=model, rep_ID=str(rep_ID))



  with open('finish_time.txt', 'a') as f:
          seconds = time.time()
          local_time = time.ctime(seconds)
          f.write('FINISH!! ')
          f.write(local_time)
          f.write('\n')

  # if len(sys.argv)>=3:
  #   projectName =  str(sys.argv[1])
  #   i = 2
  #   while i<len(sys.argv):
  #     sweepID = str(sys.argv[i])
  #     sweep_agent = 'cphnycu/' + projectName + '/' + sweepID
  #     wandb.agent(sweep_agent, train)
  #     #wandb.agent(sweep_agent, train, count=1)
  #     i=i+1
  # else:
  #   print('empty')


In [19]:
import torch.nn as nn

class NIRS_ANN(nn.Module):
    def __init__(self, Hb_num, ch, time_point, class_num):
        super(NIRS_ANN, self).__init__()
        input_dim = Hb_num * ch * time_point

        self.fulcon1 = nn.Linear(input_dim, 20, bias=True)
        self.act3 = nn.ReLU()
        self.fulcon2 = nn.Linear(20, 10, bias=True)
        self.act4 = nn.ReLU()
        self.classifier = nn.Linear(10, class_num, bias=True)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input: (batch_size, Hb_num * ch * time_point)
        x = self.fulcon1(x)
        x = self.act3(x)
        x = self.fulcon2(x)
        x = self.act4(x)
        x = self.classifier(x)

        return x


In [20]:
class NIRS_CNN(nn.Module):
    # https://www.frontiersin.org/articles/10.3389/fnrgo.2023.994969/full
    # Benchmarking framework for machine learning classification from fNIRS data
    def __init__(self, Hb_num, ch, time_point=334, class_num=3):
        super(NIRS_CNN, self).__init__()
        # bs, 1, channel, sample
        self.conv1 = nn.Conv2d(Hb_num, 4, (1, 10), padding=(0, 5),stride=(1, 2)) #更改
        self.MaxPool1 = nn.MaxPool2d((1, 2))
        self.act1 = nn.ReLU()
        self.conv2 = nn.Conv2d(4, 4, (1, 5), padding=(0, 2),stride=(1, 2)) #更改
        self.MaxPool2 = nn.MaxPool2d((1, 2))
        self.act2 = nn.ReLU()
        self.fulcon1 = nn.Linear(4*ch*math.ceil(time_point/16), 20, bias=True) # 更改
        self.act3 = nn.ReLU()
        self.fulcon2 = nn.Linear(20, 10, bias=True) # 更改
        self.act4 = nn.ReLU()
        self.classifier = nn.Linear(10, class_num, bias=True) # 更改

        self.linear = 4*ch*math.ceil(time_point/16)

    def forward(self, x):
        x = self.conv1(x)
        x = self.MaxPool1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.MaxPool2(x)
        x = x.view(-1, self.linear) # 更改
        x = self.fulcon1(x)
        x = self.act3(x)
        x = self.fulcon2(x)
        x = self.act4(x)
        x = self.classifier(x)

        #x = self.softmax(x)
        return x

In [35]:
my_config = config(
  Hb = "HbO+HbR_2layer",
  # Hb = "HbO",
  acti_fun = "elu",
  architecture = "EEGNet",
  # architecture = "NIRS_transformer",
  batch_size = 16,
  con1_type = "conv",
  dataset = "MA",
  duration = "25s",
  epochs = 10,
  kernel_size = 64,
  learning_rate = 0.001,
  optimizer = "adam",
  pooling_type = "avgPool",
  scheme = "SIFT",
  sep_conv_size = 16,
  sub = "1",
  weight_decay = 0.0001

)

In [ ]:
train(my_config)

In [ ]:
# for m in ["NIRS_transformer"]:
# # for m in ["NIRS_ANN", "NIRS_CNN"]:
# # for m in ["EEGNet","NIRS_transformer", "NIRS_ANN", "NIRS_CNN"]:
#   my_config.architecture=m
#   for hb in ["HbO+HbR_2layer"]:
#   # for hb in ["HbO+HbR_2layer","HbO"]:
#     my_config.Hb=hb
#     # for d in ["MA"]:
#     for d in ["preprocess","MA"]:
#       my_config.dataset=d
#       for s in ["SD","SI","SIFT"]:
#         my_config.scheme=s
#         print("----------------------------------------------------")
#         print("Hb: ",hb)
#         print("dataset: ",d)
#         print("scheme: ",s)
#         print("EEGNet: ",m)
#         print("----------------------------------------------------")
#         train(my_config)